In [1]:
from selenium import webdriver
import os
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
from datetime import datetime
import db_multiproc_calc_neutral2 as dmc

rf_rate = 0.0475
k_do_download = False

cwd = os.getcwd()

executable_path = os.path.join(cwd,'chromedriver.exe')
display(executable_path)

date_str = datetime.now().strftime("%Y-%m-%d")
#date_str = '2023-03-22'
file_date_str = date_str

download_path = os.path.join(cwd, 'local_download', file_date_str)
file_date_str

'C:\\Users\\mcbri\\PycharmProjects\\futureDataCapture\\chromedriver.exe'

'2023-03-22'

In [2]:
def get_iv_data_set(sym, file_date_str, barchart_datestr):

    exp_iv_date = pd.read_csv(os.path.join('local_download',file_date_str, f'{sym}_{file_date_str}.txt'))
    exp_date_str = exp_iv_date.index.values[0]
    exp_file_date_str = pd.to_datetime(exp_date_str).strftime('%m_%d_%y')

    iv_str = exp_iv_date.values[0][0]
    all_iv = pd.to_numeric(iv_str.replace('%',''))/100

    px_data_path = os.path.join('local_download', file_date_str, f'{sym}_daily_historical-data-{barchart_datestr}.csv')
    opx_data_path = os.path.join('local_download', file_date_str, f'{sym}-options-american-options-exp-{exp_file_date_str}-show-all-%futuresoptionsview%-daily-{barchart_datestr}.csv')
    gr_data_path = os.path.join('local_download', file_date_str, f'{sym}-volatility-greeks-exp-{exp_file_date_str}-show-all-{barchart_datestr}.csv')

    # read the options price data
    fpx = pd.read_csv(px_data_path, parse_dates=['Time']).dropna()
    fpx['date'] = fpx['Time'].map(lambda x:pd.to_datetime(x))
    fpx = fpx.drop(columns=['Time'])
    fpx=fpx.set_index(['date']).sort_index()
    close_px = fpx.loc[barchart_datestr, :]['Last']

    # read in the option prices and transform
    opx = pd.read_csv(opx_data_path)
    opx = opx.iloc[:-1,:]
    opx['call_put']=opx['Strike'].map(lambda x:x[-1])
    opx['num_strike']=opx['Strike'].map(lambda x:pd.to_numeric(x[:-1].replace('-','.').replace(',','')))
    opx=opx.drop(labels=['Strike', 'Delta', 'Prev Open', 'Prev High', 'Prev Low', 'Prev Change', 'Premium'], axis=1)
    opx = opx.rename(columns={'num_strike': 'Strike',
                              'Prev Volume':'Volume',
                              'Open Interest':'open_interest',
                              'Last':'mean_price'})
    opx = opx.set_index(['Strike', 'call_put'])

    # read in the greeks and transform
    gr = pd.read_csv(gr_data_path)
    gr = gr.iloc[:-1,:]

    gr['symbol']=sym
    gr['option_expiration']=pd.to_datetime(exp_date_str)
    gr['Strike']=gr['Strike'].map(lambda x:pd.to_numeric(x))
    gr['IV']=pd.to_numeric(gr['IV'].str[:-1])/100
    #RWM gr['IV']=all_iv
    gr['Type'] = gr['Type'].map({'Put':'P', 'Call':'C'})
    gr=gr.drop(labels=['IV Skew', 'Time', 'Last'], axis=1)
    gr=gr.rename(columns={'Type':'call_put'})
    gr=gr.set_index(['Strike', 'call_put'])

    # concat and make columns lowercase
    t=pd.concat([opx, gr], axis=1)
    t['close_px'] = close_px
    t = t.reset_index()
    t.columns = map(str.lower, t.columns)

    return t

def generate_neutral_futures(sym, file_date_str, barchart_datestr):

    t = get_iv_data_set(sym, file_date_str, barchart_datestr)

    df_iv = t.dropna()
    query_date = barchart_datestr
    root_symbol = sym
    #RWM df_iv['iv'] = all_iv

    unique_strikes = sorted(t['strike'])
    spot_prices = np.array(dmc.calc_spot_price_levels(unique_strikes), dtype=float)

    agg_neutral, df_greeks_by_strike, exp_neutral = \
        dmc.calc_daily_neutral_values( \
            df_iv,
            query_date,
            rf_rate,
            root_symbol,
            spot_prices)

    agg_neutral['Close'] = t['close_px'].values[0]
    agg_neutral['Symbol'] = sym
    return [agg_neutral, df_greeks_by_strike, exp_neutral]

In [3]:
def bar_chart_get_driver(executable_path, download_path):
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_experimental_option("prefs", {
        "download.default_directory": download_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })

    driver = webdriver.Chrome(executable_path=executable_path, options=chromeOptions)
    return driver

def bar_chart_login(driver):
    driver.get("https://www.barchart.com/login")
    driver.find_element(By.NAME, "email").click()
    driver.find_element(By.NAME, "email").send_keys("mcbride1689@gmail.com")
    driver.find_element(By.NAME, "password").click()
    driver.find_element(By.NAME, "password").send_keys("d5nJSz57A8C5")
    driver.find_element_by_class_name('login-button').click()
    return

def bar_chart_download(driver, date_str, sym, download=False):

    option_price_url = f"https://www.barchart.com/futures/quotes/{sym}/options?futuresOptionsTime=daily&moneyness=allRows"
    option_greek_url = f'https://www.barchart.com/futures/quotes/{sym}/volatility-greeks?moneyness=allRows'
    future_price_url = f'https://www.barchart.com/futures/quotes/{sym}/historical-download'
    driver.get(future_price_url)

    # download the price history
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, ".bc-price-history-checkboxes .checkbox").click()
    time.sleep(1)
    if download:
        driver.find_element(By.CSS_SELECTOR, ".add").click()
        time.sleep(2)

    # download the options prices
    driver.get(option_price_url)
    time.sleep(2)

    # get the expiration of the contract from the option price page
    temp_name = "#main-content-column > div > div:nth-child(4) > div > div:nth-child(1) > div > strong:nth-child(2)"
    expiration = driver.find_element(By.CSS_SELECTOR, temp_name).text
    temp_name = "#main-content-column > div > div:nth-child(4) > div > div.column.small-12.medium-4.text-medium-up-center > div > strong"
    iv_all = driver.find_element(By.CSS_SELECTOR, temp_name).text
    expiration = 'expiration,iv\n'+expiration+','+iv_all

    filename=os.path.join(download_path, f"{sym}_{date_str}.txt")
    with open(filename, "w") as text_file:
        print(expiration, file=text_file)

    time.sleep(2)
    if download:
        driver.find_element(By.CSS_SELECTOR, ".toolbar-button > span").click()
        time.sleep(2)

    # Get the option greeks
    time.sleep(2)
    driver.get(option_greek_url)
    time.sleep(3)
    if download:
        driver.find_element(By.CSS_SELECTOR, ".toolbar-button > span").click()
        time.sleep(3)
    return

In [4]:
all_sym = ['GCJ23', 'CLK23', 'NGJ23', 'ZCK23', 'ZWK23', 'XSK23', 'E6M23', 'ZNM23']
if k_do_download:
    driver = bar_chart_get_driver(executable_path, download_path, k_do_download)
    bar_chart_login(driver)
    for sym in all_sym:
        bar_chart_download(driver, date_str, sym)
    driver.quit()

In [5]:
barchart_datestr = pd.to_datetime(file_date_str).strftime('%m-%d-%Y')

In [6]:
temp = []
for sym in all_sym:
    print(sym)
    temp.append(generate_neutral_futures(sym, file_date_str, barchart_datestr))

GCJ23


ValueError: Unable to parse string "03/28/23" at position 0

In [ ]:
agg = [x[1] for x in temp]
temp_out = pd.concat(agg)
oi_delta = temp_out['open_interest_delta']
oi_put_call_ratio = oi_delta.groupby(['date', 'symbol', 'call_put']).sum().unstack('call_put')
oi_put_call_ratio['Delta-adj-Put-call-Ratio']= np.abs(oi_put_call_ratio['P'] / oi_put_call_ratio['C'])

In [ ]:
pcr = oi_put_call_ratio.reset_index()[['symbol', 'Delta-adj-Put-call-Ratio']].set_index('symbol')

In [ ]:
temp_iv_data = []
for sym in all_sym:
    df_iv = get_iv_data_set(sym, file_date_str, barchart_datestr)
    df_iv_temp = df_iv[['symbol', 'option_expiration', 'close_px']].tail(1)
    temp_iv_data.append(df_iv_temp)
closepx_table = pd.concat(temp_iv_data).set_index('symbol')
closepx_table = closepx_table.rename(columns={'close_px':'Futures Close', 'option_expiration':'Opex Date'})

In [ ]:
expn = [x[0] for x in temp]
final_output = pd.concat(expn).loc[-1,['date', 'symbol', 'delta_neutral', 'gamma_neutral' ]].set_index('symbol')
final_output = pd.concat([closepx_table, final_output, pcr], axis=1)
final_output

In [ ]:
final_output.to_csv('finalout.csv')
final_output